In [47]:
import pandas as pd

modalities_filepath = 'all.parquet'
cord_filepath = '2022-02-07-cord19.parquet'

modalities_df = pd.read_parquet(modalities_filepath)
cord_df = pd.read_parquet(cord_filepath)

In [49]:
modalities_df.tail()

,img,label,caption,width,height,prediction,img_path
524134,PMC7153745supplementaryfigsplit_978-90-368-111...,None,,1300,346,gra,cord19-uic/PMC7153745/supplementary/figsplit_9...
524135,PMC7153745supplementaryfigsplit_978-90-368-111...,None,,2026,1180,oth,cord19-uic/PMC7153745/supplementary/figsplit_9...
524136,PMC7153745supplementaryfigsplit_978-90-368-111...,None,,968,833,gra.his,cord19-uic/PMC7153745/supplementary/figsplit_9...
524137,PMC7153745supplementaryfigsplit_978-90-368-111...,None,,961,353,oth,cord19-uic/PMC7153745/supplementary/figsplit_9...
524138,PMC7153745supplementaryfigsplit_978-90-368-111...,None,,961,366,mol,cord19-uic/PMC7153745/supplementary/figsplit_9...


# Create a dictionary of modalities with PMCID as key

In [67]:
modalities_df.reset_index()
pmc_modalities_df = modalities_df[modalities_df.img_path.str.contains('cord19-uic')]
pmc_modalities_df['pmcid'] = pmc_modalities_df.apply(lambda x: x.img_path.split('/')[1], axis=1)
grouped = pmc_modalities_df.groupby('pmcid')['prediction']
modalities_dict = { x: list(y.values) for x, y in grouped}

# there are books in the collection!!!
# print(len(modalities_dict['PMC7153745']))

<ipython-input-67-701f42362f49>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pmc_modalities_df['pmcid'] = pmc_modalities_df.apply(lambda x: x.img_path.split('/')[1], axis=1)


# Join arrays in the dataframe

In [73]:
def get_modalities(pmcid, mod_dict):
  if pmcid in mod_dict:
    return ' '.join(mod_dict[pmcid])
  else:
    return ''

cord_df['modalities'] = cord_df.apply(lambda x: get_modalities(x.pmcid, modalities_dict), axis=1)

In [80]:
keys_modalities = set(modalities_dict.keys())
keys_cord = set(list(cord_df.pmcid.values))

intersection = keys_modalities.intersection(keys_cord)
intersection

{'PMC7114679',
 'PMC7129684',
 'PMC5035956',
 'PMC7109999',
 'PMC7117007',
 'PMC6439455',
 'PMC7106367',
 'PMC7126179',
 'PMC7126208',
 'PMC7140268',
 'PMC1087510',
 'PMC7108202',
 'PMC7111915',
 'PMC7153742',
 'PMC6291621',
 'PMC6201323',
 'PMC6981968',
 'PMC2682425',
 'PMC5801420',
 'PMC5127025',
 'PMC7125693',
 'PMC7120794',
 'PMC7105191',
 'PMC7149799',
 'PMC7108074',
 'PMC7115380',
 'PMC6999894',
 'PMC5896402',
 'PMC7086586',
 'PMC6348445',
 'PMC6057575',
 'PMC7122368',
 'PMC1559929',
 'PMC7103344',
 'PMC7123404',
 'PMC2912257',
 'PMC7130735',
 'PMC7112831',
 'PMC7122034',
 'PMC7035790',
 'PMC7117015',
 'PMC7112051',
 'PMC7112888',
 'PMC6466079',
 'PMC4644801',
 'PMC7127015',
 'PMC7106405',
 'PMC7125814',
 'PMC4690872',
 'PMC5035509',
 'PMC7124226',
 'PMC7125908',
 'PMC6304313',
 'PMC7110128',
 'PMC6758846',
 'PMC3407925',
 'PMC3356205',
 'PMC2843722',
 'PMC3978084',
 'PMC7111801',
 'PMC7094655',
 'PMC7120199',
 'PMC3155236',
 'PMC5979549',
 'PMC3589764',
 'PMC3542382',
 'PMC71222

In [81]:
cord_df[cord_df.pmcid=='PMC7114679']

,cord_uid,source_x,title,abstract,publish_time,journal,authors,url,pmcid,modalities
709226,31yznyut,Elsevier; Medline; PMC,Canine Coronavirus: Not Only an Enteric Pathogen,This article reviews the currently available l...,2011-09-29,Vet Clin North Am Small Anim Pract,"Decaro, Nicola; Buonavoglia, Canio",https://www.ncbi.nlm.nih.gov/pubmed/22041207/;...,PMC7114679,pho pho mic.lig mic.lig mic.lig mic.lig pho ph...


In [87]:
cord_df.to_parquet('./2022-02-07-cord19-wmodalities.parquet')